In [1]:

%matplotlib qt5

from modsim import *

def make_system(beta, gamma):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    B0 = beta
    init = State(S=89, I=1, R=0)
    init /= sum(init)

    t0 = 0
    t_end = 7 * 14

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma, B0=B0)

def update1(state, system):
    """Update the SIR model.
    
    state: State with variables S, I, R
    system: System with beta and gamma
    
    returns: State object
    """
    s, i, r = state

    infected = system.beta * i * s    
    recovered = system.gamma * i
    
    s -= infected
    i += infected - recovered
    r += recovered
    
    return State(S=s, I=i, R=r)

def run_simulation(system, update_func, update_beta):
    """Runs a simulation of the system.
    
    Add three Series objects to the System: S, I, R
    
    system: System object
    update_func: function that updates state
    """
    S = TimeSeries()
    I = TimeSeries()
    R = TimeSeries()
    B = TimeSeries()

    state = system.init
    t0 = system.t0
    S[t0], I[t0], R[t0] = state
    B[t0] = system.B0
    
    for t in linrange(system.t0, system.t_end):
        state = update_func(state, system)
        S[t+1], I[t+1], R[t+1] = state
        update_beta(system, slope, min_beta = 0.1, I = I[t + 1])
        B[t+1] = system.beta
    
    system.S = S
    system.I = I
    system.R = R
    system.B = B
    
    
def update_beta(system, slope, I, min_beta = 0.1):
    system.beta = slope * I + system.B0
    if (system.beta < min_beta):
        system.beta = min_beta
    return system.beta
    
def plot_results(S, I, R):
    """Plot the results of a SIR model.

    S: TimeSeries
    I: TimeSeries
    R: TimeSeries
    """
    plot(S, '--', color='blue', label='Susceptible')
    plot(I, '-', color='red', label='Infected')
    plot(R, ':', color='green', label='Recovered')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of population')

def calc_total_infected(system):
    """Fraction of population infected during the simulation.
    
    system: System object with results.
    
    returns: fraction of population
    """
    frame = system.results
    return frame.S[system.t0] - frame.S[system.t_end]

def add_immunization(system, fraction):
    """Immunize a fraction of the population.
    
    Moves the given fraction from S to R.
    
    system: System object
    fraction: number from 0 to 1
    """
    system.init.S -= fraction
    system.init.R += fraction



In [2]:
#state = update1(init, system)
tc = 1      # time between contacts in days 
tr = 4      # recovery time in days

beta = 1 / tc      # contact rate in per day
gamma = 1 / tr     # recovery rate in per day

slope = -1


system = make_system(beta, gamma)
run_simulation(system, update1, update_beta)
plot_results(system.S, system.I, system.R)

newfig()
plot(system.I, system.B, 'bo')



"""
for x in linspace(0,2,101):
    plot(system.I, system.beta, 'bo')
    update_beta(system, slope)
"""

"\nfor x in linspace(0,2,101):\n    plot(system.I, system.beta, 'bo')\n    update_beta(system, slope)\n"

In [3]:
counter = 0
slope_array = linspace(start = -60, stop = 0, num = 60)
beta_array = linspace(start = 0.2, stop = 2, num = 10)
newfig()

colors = ['red', 'yellow', 'green', 'blue', 'magenta', 'cyan', 'black', 'pink', 'brown', 'gray']

for b0 in beta_array:
    for s in slope_array:

        slope = s
        infected = TimeSeries()

        system = make_system(b0, gamma)
        run_simulation(system, update1, update_beta)
        plot(s, (system.S[system.t0] - system.S[system.t_end]), 'o', color = colors[counter], label = '%s initial beta' %b0)
        decorate(xlim = [-60, 45], xlabel = 'slope values', ylabel = '% of population infected')
        infected[s] = system.S[system.t0] - system.S[system.t_end]
    counter += 1


